# **1. Install & Import Libraries**

In [1]:
import torch
print(torch.cuda.get_device_name(0))


Tesla T4


In [2]:
!pip install kagglehub


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import kagglehub


# **2. Dogs vs Cats Dataset**

In [4]:
path = kagglehub.dataset_download(
    "bhavikjikadara/dog-and-cat-classification-dataset"
)
print("Dataset path:", path)


Using Colab cache for faster access to the 'dog-and-cat-classification-dataset' dataset.
Dataset path: /kaggle/input/dog-and-cat-classification-dataset


# **3. Common Transforms**

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


# **4. Load Dogs vs Cats Data**

In [6]:
dogcat_dataset = datasets.ImageFolder(root=path, transform=transform)

train_size = int(0.8 * len(dogcat_dataset))
val_size = len(dogcat_dataset) - train_size

dogcat_train, dogcat_val = random_split(
    dogcat_dataset, [train_size, val_size]
)

dogcat_train_loader = DataLoader(dogcat_train, batch_size=32, shuffle=True)
dogcat_val_loader = DataLoader(dogcat_val, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# **5. CNN Architecture**

In [7]:
class CNNModel(nn.Module):
    def __init__(self, activation, num_classes):
        super().__init__()

        act = {
            "relu": nn.ReLU(),
            "tanh": nn.Tanh(),
            "leaky": nn.LeakyReLU(0.1)
        }[activation]

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            act,
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            act,
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            act,
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            act,
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


# **6. Weight Initialization**

In [8]:
def init_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(m.weight)
            elif init_type == "random":
                nn.init.normal_(m.weight, 0, 0.02)


# **7. Optimizer Selector**

In [9]:
def get_optimizer(name, model):
    if name == "sgd":
        return optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif name == "adam":
        return optim.Adam(model.parameters(), lr=0.001)
    elif name == "rmsprop":
        return optim.RMSprop(model.parameters(), lr=0.001)


In [10]:
search_size = 5000

dogcat_search, _ = torch.utils.data.random_split(
    dogcat_train,
    [search_size, len(dogcat_train) - search_size]
)

dogcat_search_loader = DataLoader(
    dogcat_search,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)


# **8. Training & Evaluation Function**

In [11]:
def train_model(model, train_loader, val_loader, optimizer, epochs=5):
    criterion = nn.CrossEntropyLoss()
    best_acc = 0

    for _ in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                pred = model(x).argmax(1)
                total += y.size(0)
                correct += (pred == y).sum().item()

        acc = correct / total
        best_acc = max(best_acc, acc)

    return best_acc


# **9. Evaluating : Dogs vs Cats (with 27 models)**

In [12]:
activations = ["relu", "tanh", "leaky"]
inits = ["xavier", "kaiming", "random"]
optimizers = ["sgd", "adam", "rmsprop"]

best_dogcat_acc = 0

for act in activations:
    for init in inits:
        for opt in optimizers:
            model = CNNModel(act, 2).to(device)
            init_weights(model, init)
            optimizer = get_optimizer(opt, model)

            acc = train_model(
              model,
              dogcat_search_loader,
              dogcat_val_loader,
              optimizer,
              epochs=1
           )

            if acc > best_dogcat_acc:
                best_dogcat_acc = acc
                torch.save(model.state_dict(), "best_cnn_dogcat.pth")

print("Best Dog vs Cat CNN Accuracy:", best_dogcat_acc)


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings

Best Dog vs Cat CNN Accuracy: 1.0


In [13]:
best_dogcat_acc = 0
best_config = None

if acc > best_dogcat_acc:
    best_dogcat_acc = acc
    best_config = (act, init, opt)
    torch.save(model.state_dict(), "best_cnn_dogcat.pth")


In [14]:
best_act, best_init, best_opt = best_config

best_model = CNNModel(best_act, 2).to(device)
init_weights(best_model, best_init)
optimizer = get_optimizer(best_opt, best_model)

final_acc = train_model(
    best_model,
    dogcat_train_loader,
    dogcat_val_loader,
    optimizer,
    epochs=10
)

print("Final Dog vs Cat Accuracy:", final_acc)

/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Final Dog vs Cat Accuracy: 1.0


# **10. Evaluating : CIFAR-10 (with 27 models)**

In [17]:
activations = ["relu", "tanh", "leaky"]
inits = ["xavier", "kaiming", "random"]
optimizers = ["sgd", "adam", "rmsprop"]

best_cifar_acc = 0
best_cifar_config = None

for act in activations:
    for init in inits:
        for opt in optimizers:
            print(f"Training: {act}, {init}, {opt}")

            model = CNNModel(act, 10).to(device)
            init_weights(model, init)
            optimizer = get_optimizer(opt, model)

            acc = train_model(
                model,
                cifar_train_loader,
                cifar_test_loader,
                optimizer,
                epochs=1
            )

            if acc > best_cifar_acc:
                best_cifar_acc = acc
                best_cifar_config = (act, init, opt)
                torch.save(model.state_dict(), "best_cnn_cifar10.pth")

print("Best CIFAR-10 Accuracy (Search):", best_cifar_acc)
print("Best Config:", best_cifar_config)


Training: relu, xavier, sgd
Training: relu, xavier, adam
Training: relu, xavier, rmsprop
Training: relu, kaiming, sgd
Training: relu, kaiming, adam
Training: relu, kaiming, rmsprop
Training: relu, random, sgd
Training: relu, random, adam
Training: relu, random, rmsprop
Training: tanh, xavier, sgd
Training: tanh, xavier, adam
Training: tanh, xavier, rmsprop
Training: tanh, kaiming, sgd
Training: tanh, kaiming, adam
Training: tanh, kaiming, rmsprop
Training: tanh, random, sgd
Training: tanh, random, adam
Training: tanh, random, rmsprop
Training: leaky, xavier, sgd
Training: leaky, xavier, adam
Training: leaky, xavier, rmsprop
Training: leaky, kaiming, sgd
Training: leaky, kaiming, adam
Training: leaky, kaiming, rmsprop
Training: leaky, random, sgd
Training: leaky, random, adam
Training: leaky, random, rmsprop
Best CIFAR-10 Accuracy (Search): 0.6085
Best Config: ('leaky', 'kaiming', 'adam')


In [18]:
best_act, best_init, best_opt = best_cifar_config

final_cifar_model = CNNModel(best_act, 10).to(device)
init_weights(final_cifar_model, best_init)
optimizer = get_optimizer(best_opt, final_cifar_model)

final_cifar_acc = train_model(
    final_cifar_model,
    cifar_train_loader,
    cifar_test_loader,
    optimizer,
    epochs=10
)

torch.save(final_cifar_model.state_dict(), "final_best_cnn_cifar10.pth")

print("Final CIFAR-10 Accuracy:", final_cifar_acc)


Final CIFAR-10 Accuracy: 0.7422


In [19]:
print("CIFAR-10 Results")
print("----------------")
print("Best Config :", best_cifar_config)
print("Final Accuracy :", final_cifar_acc)


CIFAR-10 Results
----------------
Best Config : ('leaky', 'kaiming', 'adam')
Final Accuracy : 0.7422


# **12. ResNet-18 (Transfer Learning – Both Datasets)**

### **(a) ResNet-18 on CAT vs Dog**

In [20]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, 2)
resnet = resnet.to(device)

optimizer = optim.Adam(resnet.parameters(), lr=0.0001)

dogcat_resnet_acc = train_model(
    resnet,
    dogcat_train_loader,
    dogcat_val_loader,
    optimizer
)

torch.save(resnet.state_dict(), "resnet18_dogcat.pth")


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


### **(b) ResNet-18 on CIFAR-10**

In [22]:
from torchvision import models
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Linear(resnet.fc.in_features, 10)

resnet = resnet.to(device)

# Optimizer
optimizer = optim.Adam(resnet.parameters(), lr=0.0001)

# Train
cifar_resnet_acc = train_model(
    resnet,
    cifar_train_loader,
    cifar_test_loader,
    optimizer,
    epochs=5
)

# Save weights
torch.save(resnet.state_dict(), "resnet18_cifar10.pth")

print("ResNet-18 CIFAR-10 Accuracy:", cifar_resnet_acc)


ResNet-18 CIFAR-10 Accuracy: 0.9338


## **Final Comaprison Output**

In [23]:
print("Dogs vs Cats:")
print("Best CNN :", best_dogcat_acc)
print("ResNet18 :", dogcat_resnet_acc)

print("\nCIFAR-10:")
print("Best CNN :", best_cifar_acc)
print("ResNet18 :", cifar_resnet_acc)


Dogs vs Cats:
Best CNN : 1.0
ResNet18 : 1.0

CIFAR-10:
Best CNN : 0.6085
ResNet18 : 0.9338
